![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/fuzzy-group.png)

# Fuzzy Grouping

Unprepared data often represents the same entity with multiple values; examples include different spellings, varying capitalizations, and abbreviations. This is common when working with data gathered from multiple sources or through human input. One way to canonicalize and reconcile these variants is to use Data Prep's fuzzy_group_column (also known as "text clustering") functionality.

Data Prep inspects a column to determine clusters of similar values. A new column is added in which clustered values are replaced with the canonical value of its cluster, thus significantly reducing the number of distinct values. You can control the degree of similarity required for values to be clustered together, override canonical form, and set clusters if automatic clustering did not provide the desired results.

Let's explore the capabilities of `fuzzy_group_column` by first reading in a dataset and inspecting it.

In [ ]:
import azureml.dataprep as dprep

In [ ]:
dflow = dprep.read_json(path='../data/json.json')
dflow.head(5)

As you can see above, the column `inspections.business.city` contains several forms of the city name "San Francisco".
Let's add a column with values replaced by the automatically detected canonical form. To do so call fuzzy_group_column() on an existing Dataflow:

In [ ]:
dflow_clean = dflow.fuzzy_group_column(source_column='inspections.business.city',
                                       new_column_name='city_grouped',
                                       similarity_threshold=0.8,
                                       similarity_score_column_name='similarity_score')
dflow_clean.head(5)

The arguments `source_column` and `new_column_name` are required, whereas the others are optional.
If `similarity_threshold` is provided, it will be used to control the required similarity level for the values to be grouped together.
If `similarity_score_column_name` is provided, a second new column will be added to show similarity score between every pair of original and canonical values.

In the resulting data set, you can see that all the different variations of representing "San Francisco" in the data were normalized to the same string, "San Francisco".

But what if you want more control over what gets grouped, what doesn't, and what the canonical value should be? 

To get more control over grouping, canonical values, and exceptions, you need to use the `FuzzyGroupBuilder` class.
Let's see what it has to offer below:

In [ ]:
builder = dflow.builders.fuzzy_group_column(source_column='inspections.business.city',
                                            new_column_name='city_grouped',
                                            similarity_threshold=0.8,
                                            similarity_score_column_name='similarity_score')

In [ ]:
# calling learn() to get fuzzy groups
builder.learn()
builder.groups

Here you can see that `fuzzy_group_column` detected one group with four values that all map to "San Francisco" as the canonical value.
You can see the effects of changing the similarity threshold next:

In [ ]:
builder.similarity_threshold = 0.9
builder.learn()
builder.groups

Now that you are using a similarity threshold of `0.9`, two distinct groups of values are generated.

Let's tweak some of the detected groups before completing the builder and getting back the Dataflow with the resulting fuzzy grouped column.

In [ ]:
builder.similarity_threshold = 0.8
builder.learn()
groups = builder.groups
groups

In [ ]:
# change the canonical value for the first group
groups[0]['canonicalValue'] = 'SANFRAN'
duplicates = groups[0]['duplicates']
# remove the last duplicate value from the cluster
duplicates = duplicates[:-1]
# assign modified duplicate array back
groups[0]['duplicates'] = duplicates
# assign modified groups back to builder
builder.groups = groups
builder.groups

Here, the canonical value is modified to be used for the single fuzzy group and removed 'S.F.' from this group's duplicates list.

You can mutate the copy of the `groups` list from the builder (be careful to keep the structure of objects inside this list). After getting the desired groups in the list, you can update the builder with it.

Now you can get a dataflow with the FuzzyGroup step in it.

In [ ]:
dflow_clean = builder.to_dataflow()

df = dflow_clean.to_pandas_dataframe()
df